In [2]:
pip install Keras-Preprocessing


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#To Create a DataFrame 
import pandas as pd
#Use of arrays in Python
import numpy as np
#For Plotting the accuracy
import matplotlib.pyplot as plt
import seaborn as sns
#Natural Language Toolkit for NLP related work
import nltk

# !conda install keras
#Tokenizing the sentences to word
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
#Padding the sentences
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')
sns.set()
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
data = pd.read_csv('IMDB Dataset.csv')
data = pd.DataFrame(data)
data.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [5]:
corpus_1 = []
for text in data['review']:
    words = [word.lower() for word in word_tokenize(text)]
    corpus_1.append(words)

In [6]:
number_of_words=len(corpus_1)
print(number_of_words)

50000


In [7]:
X_train = data.review[:]
X_train

0        One of the other reviewers has mentioned that ...
1        A wonderful little production. <br /><br />The...
2        I thought this was a wonderful way to spend ti...
3        Basically there's a family where a little boy ...
4        Petter Mattei's "Love in the Time of Money" is...
                               ...                        
49995    I thought this movie did a down right good job...
49996    Bad plot, bad dialogue, bad acting, idiotic di...
49997    I am a Catholic taught in parochial elementary...
49998    I'm going to have to disagree with the previou...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 50000, dtype: object

In [8]:
y_train= data.sentiment[:]
y_train

0        positive
1        positive
2        positive
3        negative
4        positive
           ...   
49995    positive
49996    negative
49997    negative
49998    negative
49999    negative
Name: sentiment, Length: 50000, dtype: object

In [9]:
# Test One

In [10]:
test_data= pd.read_csv('IMDB Dataset.csv')
test_data = pd.DataFrame(test_data)


corpus_2 = []
for text in test_data['review']:
    words = [word.lower() for word in word_tokenize(text)]
    corpus_2.append(words)

In [11]:
X_test= test_data.review[:]
y_test = test_data.sentiment[:]

# Tokeinzing and Padding the words

In [12]:
tokenizer = Tokenizer(number_of_words)
tokenizer.fit_on_texts(X_train)


In [13]:
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=128, truncating='post', padding='post')

In [14]:
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=128, truncating='post', padding='post')

In [15]:
#I Dont think we need LabelEncoder() Here..!!!

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [16]:
model = Sequential()

model.add(Embedding(input_dim=number_of_words, output_dim=100,
                   input_length=128, trainable=True))
model.add(LSTM(100,dropout=0.1,return_sequences=True))
model.add(LSTM(100,dropout=0.1))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 128, 100)          5000000   
                                                                 
 lstm (LSTM)                 (None, 128, 100)          80400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 5,160,901
Trainable params: 5,160,901
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history=model.fit(X_train, y_train,epochs=5, batch_size=61, validation_data=(X_test,y_test))

Epoch 1/5
820/820 [==============================] - 710s 861ms/step - loss: 0.4375 - accuracy: 0.7998 - val_loss: 0.2468 - val_accuracy: 0.9138
Epoch 2/5
820/820 [==============================] - 698s 852ms/step - loss: 0.2483 - accuracy: 0.9067 - val_loss: 0.1493 - val_accuracy: 0.9527
Epoch 3/5
820/820 [==============================] - ETA: 0s - loss: 0.1610 - accuracy: 0.9415

In [ ]:
plt.figure(figsize=(16,5))
epochs=range(1,len(history.history['accuracy'])+1)
plt.plot(epochs,history.history['loss'], 'b', label='Training Loss',color='red')
plt.plot(epochs,history.history['val_loss'], 'b', label='Validation  Loss',color='orange')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16,5))
epochs=range(1,len(history.history['accuracy'])+1)
plt.plot(epochs,history.history['accuracy'], 'b', label='Training Accuracy',color='red')
plt.plot(epochs,history.history['val_accuracy'], 'b', label='Validation  Accuracy',color='orange')
plt.legend()
plt.show()